In [ ]:
# Code for recommender
# Input files - Food DB;  User Interaction Log; Cluster mapping
# Output - a personalised recommnedation for the customer

In [4]:
import pandas as pd
import numpy as np
import io
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import StandardScaler

In [2]:
from google.colab import files
uploaded = files.upload()

Saving Food DB.json to Food DB.json


In [5]:
df = pd.read_json(io.StringIO(uploaded.get('Food DB.json').decode('utf-8')))
df.head(2)

,Name,description,sub_catagory,Veg_Non,Review,Cusine,Diet,Price,ingredients_list,Allergy,Nutrient,Disease (Bad for),Category,Season,Mood
0,Himachali Dry Raw Mango Chutney Recipe,Aam Papad (Sun Dried Mango)|Mint Leaves (Pudin...,Indian Chutney,veg,4.909474,Himachal,Vegetarian,475,"[aam papad (sun dried mango), mint leaves (pud...","[Oral Allergy Syndrome, Sugar Allergy / Intole...","[, carbs, sodium]","[, Diabetes,Obesity,Heart-related ailments,PCO...",Side dish,no season,Salt craving
1,Aamras Ki Kadhi Recipe - Mango Kadhi Recipe,Aamras|Gram flour (besan)|Turmeric powder (Hal...,Indian Curry,veg,4.873846,Gujarati Recipes﻿,Vegetarian,337,"[aamras, gram flour (besan), turmeric powder (...","[Legume Allergy, Histamine Allergy]","[, copper, sodium]","[, Blood pressure]",Lunch / Dinner,no season,Hungry


In [6]:
from google.colab import files
uploaded = files.upload()

Saving User Interaction Log.json to User Interaction Log.json


In [7]:
orders = pd.read_json(io.StringIO(uploaded.get('User Interaction Log.json').decode('utf-8')))

In [17]:
from google.colab import files
uploaded = files.upload()

Saving Cluster Mapping.json to Cluster Mapping (1).json


In [18]:
usercluster = pd.read_json(io.StringIO(uploaded.get('Cluster Mapping.json').decode('utf-8')))
usercluster.head()

,User_ID,cluster
0,0,0
1,1,0
2,2,2
3,3,2
4,4,1


In [19]:
# read dish database and scale prices
#df = pd.read_json('Food DB.json')
std_scaler = StandardScaler() 
df[['Scaled prices']] = StandardScaler().fit_transform(df[['Price']])

# read userlog
#orders = pd.read_json('User Interaction Log.json')
# read user clusters

#usercluster = pd.read_json('Cluster Mapping.json')
#usercluster.drop(usercluster.iloc[:, 1:47], axis = 1, inplace = True)
#usercluster.head()

In [20]:
# Map the cluster number in each line of user interaction log
orders = pd.merge(orders , usercluster , how='left', left_on='User_ID', right_on='User_ID')

In [21]:
# make a list of all users by cluster
cluster_zero = pd.pivot(usercluster, columns="cluster", values = 'User_ID')[0].dropna().tolist()
cluster_one = pd.pivot(usercluster, columns="cluster", values = 'User_ID')[1].dropna().tolist()
cluster_two = pd.pivot(usercluster, columns="cluster", values = 'User_ID')[2].dropna().tolist()

def get_user_cluster(current_user):
    if current_user in cluster_zero:
        return 0
    else:
        if current_user in cluster_one:
            return 1
        else:
            return 2

def get_my_dishes(current_user_cluster):
    if current_user_cluster == 0:
        return cluster_zero_dishes
    else:
        if current_user_cluster == 1:
            return cluster_one_dishes
        else: 
            return cluster_two_dishes

In [22]:
# map cluster to userlog data
cluster_zero_dishes = pd.pivot(orders, columns="cluster", values = 'Name')[0].dropna().tolist()
cluster_one_dishes = pd.pivot(orders, columns="cluster", values = 'Name')[1].dropna().tolist()
cluster_two_dishes = pd.pivot(orders, columns="cluster", values = 'Name')[2].dropna().tolist()

In [23]:
# determine popularity Scores based on user interactions
orders['Score'] = orders['Searched'] + orders['Purchased'] + orders['Liked'] 
user_interaction_count = orders.groupby('Name')['Score'].sum().reset_index()
df = pd.merge(df,user_interaction_count, how='left', left_on='Name', right_on='Name')
df['Score'].fillna(0, inplace=True)

In [24]:
# Average Reviews of all items
C= df['Review'].mean()

# Min votes required to appear in recommendation list fixed at 60% among 'Score'
m= df['Score'].quantile(0.6)

# shortlist dishes above 60%ile votes
q_items = df.copy().loc[df['Score'] >= m]

# Calculation of weighted rating based on the IMDB formula
def weighted_rating(x, m=m, C=C):
    v = x['Score']
    R = x['Review']
    return (v/(v+m) * R) + (m/(m+v) * C ) + x['Scaled prices']/1000

# Applying weighted_rating to qualified items
q_items['weighted_score'] = q_items.apply(weighted_rating, axis=1)

# Shortlisting the top rated items and popular items
top_rated_items = q_items.sort_values('weighted_score', ascending=False)
pop_items= df.sort_values('Score', ascending=False)
top_rated_items

,Name,description,sub_catagory,Veg_Non,Review,Cusine,Diet,Price,ingredients_list,Allergy,Nutrient,Disease (Bad for),Category,Season,Mood,Scaled prices,Score,weighted_score
5541,White Chocolate Pistachio Bars Recipe | White ...,White Chocolate|Butter (unsalted)|Condensed Mi...,Dessert,veg,5.000000,Continental,Vegetarian,452,"[white chocolate, butter (unsalted), condensed...",[Milk allergy / Lactose intolerance],"[maginesum, sodium, vitamin_D, protien]","[, Blood pressure]",Dessert,Summer,"Sweet Craving,Celebrate",0.453253,6.0,4.956985
764,Bathua Raita Recipe,Bathua leaves|Green Chillies|Ginger|Black Salt...,Indian Curry,veg,5.000000,North Indian Recipes,Diabetic Friendly,503,"[bathua leaves, green chillies, ginger, black ...",[Histamine Allergy],"[, sodium, vitamin_E]","[, Blood pressure, High Cholestrol]",Lunch / Dinner,no season,Hungry,0.899359,5.0,4.951221
1685,Chicken Souvlaki with Tzatziki Recipe,Chicken breasts|Extra Virgin Olive Oil|Lemon j...,Continental Food,non-veg,4.980392,Greek,High Protein Non Vegetarian,429,"[chicken breasts, extra virgin olive oil, lemo...","[Poultry Allergy, Citrus Allergy, Allium Aller...","[iron, vitamin_E, vitamin_C, , sodium, calcium]","[, High Cholestrol, Blood pressure]",Lunch / Dinner,School vacations,Hungry,0.252069,8.0,4.949791
5902,Whole Wheat Methi Garlic Naan Recipe | Oven M...,Whole Wheat Flour|Salt|Sugar|Garlic|Methi Leav...,Indian Bread,veg,4.979198,North Indian Recipes,Vegetarian,352,"[whole wheat flour, salt, sugar, garlic, methi...","[Gluten Allergy, Sugar Allergy / Intolerance, ...","[carbs, sodium, , vitamin_E]","[Diabetes,Obesity,Heart-related ailments,PCOD,...",Lunch / Dinner,no season,Healthy eating,-0.421463,8.0,4.948162
314,Homemade Tagliatelle Pasta with Capers and Bel...,All Purpose Flour (Maida)|Whole Wheat Flour|Wh...,Italian,veg,4.994135,Italian Recipes,Vegetarian,456,"[all purpose flour (maida), whole wheat flour,...","[Gluten Allergy, Allium Allergy]","[carbs, vitamin_A, sodium, vitamin_E, , vitami...","[Diabetes,Obesity,Heart-related ailments,PCOD,...",Lunch / Dinner,IPL,Bored,0.488242,5.0,4.946621
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7008,lamb rogan josh,"lamb chops or stewing lamb, vegetable oil, cas...",Indian Snack,veg,4.000000,Indian,Vegetarian,264,"[lamb chops or stewing lamb, vegetable oil, ca...","[Allium Allergy, Milk allergy / Lactose intole...","[iron, vitamin_E, , carbs, vitamin_C, sodium]","[, High Cholestrol, Diabetes,Obesity,Heart-rel...",Tea time snack,School vacations,Bored,-1.191214,8.0,4.164034
6994,koldil chicken,ith banana flower; an assamese specialty,Continental Food,veg,4.000000,Indian,Vegetarian,247,[ith banana flower; an assamese specialty],[Banana Allergy],[selenium],[],Lunch / Dinner,School vacations,Hungry,-1.339916,8.0,4.163885
6903,oat crusted chicken tenders,"chicken breast (cut into strips), salt, garlic...",Indian Snack,non-veg,4.000000,Continental,Vegetarian,596,"[chicken breast (cut into strips), salt, garli...","[Poultry Allergy, Allium Allergy, Histamine Al...","[iron, sodium, , protien]","[, Blood pressure]",Tea time snack,School vacations,Bored,1.712845,11.0,4.128809
6912,Chicken razala,"Chicken, dahi, sesame seeds, garam masala powd...",Continental Food,non-veg,4.000000,Indian,Non Vegeterian,362,"[chicken, dahi, sesame seeds, garam masala pow...","[Poultry Allergy, Seed Allergy]","[iron, , copper, vitamin_E]","[, High Cholestrol]",Lunch / Dinner,School vacations,Hungry,-0.333992,11.0,4.126762


In [25]:
# Creating soup string for each item
def create_soup(x):            
    tags = x['Name'].lower().split(', ')
    tags.extend(x['Category'].lower().split())
    tags.extend(x['sub_catagory'].lower().split())
    tags.extend(x['Cusine'].lower().split())
    tags.extend(x['Season'].lower().split())
    tags.extend(x['Mood'].lower().split())
    tags.extend(x['Diet'].lower().split())
    tags.extend(x['Veg_Non'].lower().split())
    return " ".join(sorted(set(tags), key=tags.index))

df['soup'] = df.apply(create_soup, axis=1)

In [26]:
# create count matrix and apply cosine similarity on it
count = CountVectorizer(stop_words='english')
count_matrix = count.fit_transform(df['soup'])
cosine_sim = cosine_similarity(count_matrix, count_matrix)

indices_from_title = pd.Series(df.index, index=df['Name'])

In [27]:
# Function that takes in Name of dish and outputs 10 most similar dishes 
def get_recommendations(title="", cosine_sim=cosine_sim, idx=-1):
    if idx == -1 and title != "":
        idx = indices_from_title[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:6] # change to how many every dishes you would want recommended by name
    food_indices = [i[0] for i in sim_scores]
    return food_indices

# test the code with a random dish
df.loc[get_recommendations(title="Tomato Chutney Recipe")]

,Name,description,sub_catagory,Veg_Non,Review,Cusine,Diet,Price,ingredients_list,Allergy,Nutrient,Disease (Bad for),Category,Season,Mood,Scaled prices,Score,soup
5327,Raw Tomato Chutney Recipe,Tomatoes|Green Chillies|Garlic|Methi Seeds (Fe...,Indian Chutney,veg,4.913400,South Indian Recipes,Vegetarian,406,"[tomatoes, green chillies, garlic, methi seeds...","[Nightshade Allergy, Allium Allergy]","[vitamin_C, , copper, sodium, vitamin_E]","[, Blood pressure, High Cholestrol]",Side dish,no season,Salt craving,0.050884,0.0,raw tomato chutney recipe side dish indian chu...
2367,Roasted Garlic Tomato Chutney Recipe,Garlic|Tomato|Dry Red Chilli|Tamarind|Sunflowe...,Indian Chutney,veg,4.923077,South Indian Recipes,Vegetarian,259,"[garlic, tomato, dry red chilli, tamarind, sun...","[Allium Allergy, Nightshade Allergy]","[, vitamin_C, vitamin_E, sodium, copper]","[, High Cholestrol, Blood pressure]",Side dish,no season,Salt craving,-1.234950,6.0,roasted garlic tomato chutney recipe side dish...
1028,Brahmi Chutney Recipe,Brahmi leaves|Fresh coconut|White Urad Dal (Sp...,Indian Chutney,veg,4.899351,South Indian Recipes,Vegetarian,230,"[brahmi leaves, fresh coconut, white urad dal ...",[Legume Allergy],"[, manganese, sodium, vitamin_E]","[, Blood pressure, High Cholestrol]",Side dish,no season,Salt craving,-1.488617,0.0,brahmi chutney recipe side dish indian chutney...
5355,Tomato Onion Chutney Recipe - Thakkali Vengaya...,Tomatoes|Pearl onions (Sambar Onions)|Green Ch...,Indian Chutney,veg,4.875684,South Indian Recipes,Vegetarian,310,"[tomatoes, pearl onions (sambar onions), green...","[Nightshade Allergy, Allium Allergy, Seed Alle...","[vitamin_C, , vitamin_E, sodium, copper]","[, High Cholestrol, Blood pressure]",Side dish,no season,Salt craving,-0.788844,0.0,tomato onion chutney recipe - thakkali vengaya...
3769,Tomato Onion Chutney Recipe With Coconut And Mint,Onions|Tomatoes|Green Chillies|Mint Leaves (Pu...,Indian Chutney,veg,4.647856,South Indian Recipes,Vegetarian,287,"[onions, tomatoes, green chillies, mint leaves...","[Allium Allergy, Nightshade Allergy]","[, vitamin_C, manganese, copper, vitamin_E, so...","[, High Cholestrol, Blood pressure]",Side dish,no season,Salt craving,-0.990029,2.0,tomato onion chutney recipe with coconut and m...


In [28]:
# Function to recommend based on past orders
def get_latest_user_orders(user_id, orders, num_orders=1):
    counter = num_orders
    order_indices = []
    
    for index, row in orders[['User_ID']].iterrows():
        if row.User_ID == user_id:
            counter = counter -1
            order_indices.append(index)
        if counter == 0:
            break        
    return order_indices

In [75]:
# return food_indices for accomplishing personalized recommendation using Count Vectorizer
def personalised_recomms(orders, df1, user_id, columns, comment="Based on your past orders"):
    order_indices = get_latest_user_orders(user_id, orders)
    food_ids = []
    recomm_indices = []
    
    for i in order_indices:
        food_ids.append(orders.loc[i].Name)
    for i in food_ids:
        recomm_indices.extend(get_recommendations(title=i))
        
    display(get_recomms_df(set(recomm_indices), df1, columns, comment))
    return get_recomms_df(set(recomm_indices), df1, columns, comment)


# fetch items from previously calculated top_rated_items list
def get_top_rated_items(top_rated_items, df1, columns,current_user ,comment="Top rated dishes by similar users"):
    current_user_cluster = get_user_cluster(current_user)
    my_dishes = get_my_dishes(current_user_cluster)
    food_indices = []
    
    for index, row in top_rated_items.head(20).iterrows():
        food_indices.append(indices_from_title[top_rated_items.loc[index].Name])
    
    food_indices_final = []
    
    # remove items that are not in our cluster
    for i in food_indices:
        dishname = df.iloc[i].Name
        if dishname in my_dishes:
            food_indices_final.append(i)
    
    display(get_recomms_df(food_indices_final, df1, columns, comment).head(5))
    return get_recomms_df(food_indices_final, df1, columns, comment)

# fetch items from previously calculated pop_items list
def get_popular_items(pop_items, df1, columns, current_user ,comment="Popular dishes among similar users"):
    current_user_cluster = get_user_cluster(current_user)
    my_dishes = get_my_dishes(current_user_cluster)
    food_indices = []
    for index, row in pop_items.iterrows():
        food_indices.append(indices_from_title[pop_items.loc[index].Name])
    
    food_indices_final = []
    # remove items that are not in our cluster
    for i in food_indices:
        dishname = df.iloc[i].Name
        if dishname in my_dishes:
            food_indices_final.append(i)
    
    return get_recomms_df(food_indices_final, df1, columns, comment)

In [30]:
# utility function to return df given recipie Name
def get_recomms_df(food_indices, df1, columns, comment):
    row = 0
    df = pd.DataFrame(columns=columns)
    
    for i in food_indices:
        df.loc[row] = df1[['Name', 'sub_catagory', 'Cusine', 'Diet']].loc[i]
        df.loc[row].comment = comment
        row = row+1
    return df

In [81]:
columns = ['Name', 'sub_catagory', 'Cusine', 'Diet','comment']

In [82]:
print("Are you new user or existing user?")
new_repeat = int(input("enter 1 for NEW, 0 for EXISTING: "))
#print(new_repeat)
if new_repeat == 0:
  current_user = int(input("Please enter your User ID "))
  print("We have interesting recommendations for you: ")
  personalised_recomms(orders, df, current_user, columns)
  get_top_rated_items(top_rated_items, df, columns,current_user).head(5)
else:
  display(top_rated_items[['Name', 'sub_catagory', 'Cusine', 'Diet']].head())

Are you new user or existing user?
enter 1 for NEW, 0 for EXISTING: 1


,Name,sub_catagory,Cusine,Diet
5541,White Chocolate Pistachio Bars Recipe | White ...,Dessert,Continental,Vegetarian
764,Bathua Raita Recipe,Indian Curry,North Indian Recipes,Diabetic Friendly
1685,Chicken Souvlaki with Tzatziki Recipe,Continental Food,Greek,High Protein Non Vegetarian
5902,Whole Wheat Methi Garlic Naan Recipe | Oven M...,Indian Bread,North Indian Recipes,Vegetarian
314,Homemade Tagliatelle Pasta with Capers and Bel...,Italian,Italian Recipes,Vegetarian


In [79]:
#personalised_recomms(orders, df, current_user, columns)

In [77]:
#get_top_rated_items(top_rated_items, df, columns,current_user).head(5)

In [78]:
#get_popular_items(pop_items, df, columns,current_user).head(5)